In [1]:
import tensorflow as tf
from research.object_detection.utils import dataset_util
from research.object_detection.utils import visualization_utils as viz_utils
from research.object_detection.utils import label_map_util
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

import numpy as np 
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('TkAgg')
#import tkinter 
#import ast

In [2]:
record_train = "data/muscima/training.record"

class_map = "mapping_all_classes.txt"

In [3]:
def generate_omr_dataset_from_tfrecord(filename: str):
    dataset = tf.data.TFRecordDataset(filename)

    dataset = dataset.map(
        parse_omr_element
        )

    return dataset

    
def parse_omr_element(element):

    data = {
    'image/height': tf.io.FixedLenFeature([], tf.int64),
    'image/width': tf.io.FixedLenFeature([], tf.int64),
    'image/filename': tf.io.FixedLenFeature([], tf.string),
    'image/source_id': tf.io.FixedLenFeature([], tf.string),
    #'image/key/sha256':tf.io.FixedLenFeature([], tf.string),
    'image/encoded': tf.io.FixedLenFeature([], tf.string),
    #'image/encoded': tf.io.decode_png([], channels=0),
    'image/format': tf.io.FixedLenFeature([], tf.string),
    
    'image/object/bbox/xmin': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    #'image/object/bbox/xmin': tf.io.VarLenFeature(tf.float32),
    'image/object/bbox/xmax': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'image/object/bbox/ymin': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'image/object/bbox/ymax': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    #'image/object/class/text': tf.io.FixedLenSequenceFeature([], tf.string, allow_missing=True),
    'image/object/class/text': tf.io.VarLenFeature(tf.string),
    'image/object/class/label': tf.io.FixedLenSequenceFeature([], tf.int64, allow_missing=True),
    }


    example = tf.io.parse_single_example(element, data)

    example['image/decoded'] = tf.io.decode_image(example.get('image/encoded'))

    example_np_xmin = example['image/object/bbox/xmin']
    example_np_xmax = example['image/object/bbox/xmax']
    example_np_ymin = example['image/object/bbox/ymin']
    example_np_ymax = example['image/object/bbox/ymax']
    
    example_np_bbox = tf.stack((example_np_ymin, example_np_xmin, example_np_ymax, example_np_xmax))

    example['image/bbox'] = tf.transpose(example_np_bbox)

    # get rid of irrelevant objects
    example.pop('image/encoded')
    example.pop('image/format')
    #example.pop('image/object/bbox/xmin')
    example.pop('image/object/bbox/ymin')
    example.pop('image/object/bbox/xmax')
    example.pop('image/object/bbox/ymax')

    return example


def plot_detections(image_np,
                    boxes,
                    classes, 
                    scores, 
                    category_index, 
                    figsize=(12,16),
                    image_name=None):
    image_np_with_annotations = image_np.copy()
    image_np_with_annotations = np.repeat(image_np_with_annotations, 3, 2)
    viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_annotations,
        boxes,
        classes,
        scores,
        category_index,
        use_normalized_coordinates=True,
        min_score_thresh=0.8,
        max_boxes_to_draw=scores.shape[0]
    )
    if image_name:
        plt.imsave(image_name, image_np_with_annotations)
    else:
        plt.imshow(image_np_with_annotations)
        plt.show()

In [4]:
dataset = tf.data.TFRecordDataset(record_train)
sample = dataset.take(1)
td_element = sample.get_single_element()

In [5]:
training_dataset = generate_omr_dataset_from_tfrecord(record_train)
sample = training_dataset.take(1)

In [6]:
td_element = sample.get_single_element()
print(td_element['image/object/class/text'])

SparseTensor(indices=tf.Tensor(
[[  0]
 [  1]
 [  2]
 [  3]
 [  4]
 [  5]
 [  6]
 [  7]
 [  8]
 [  9]
 [ 10]
 [ 11]
 [ 12]
 [ 13]
 [ 14]
 [ 15]
 [ 16]
 [ 17]
 [ 18]
 [ 19]
 [ 20]
 [ 21]
 [ 22]
 [ 23]
 [ 24]
 [ 25]
 [ 26]
 [ 27]
 [ 28]
 [ 29]
 [ 30]
 [ 31]
 [ 32]
 [ 33]
 [ 34]
 [ 35]
 [ 36]
 [ 37]
 [ 38]
 [ 39]
 [ 40]
 [ 41]
 [ 42]
 [ 43]
 [ 44]
 [ 45]
 [ 46]
 [ 47]
 [ 48]
 [ 49]
 [ 50]
 [ 51]
 [ 52]
 [ 53]
 [ 54]
 [ 55]
 [ 56]
 [ 57]
 [ 58]
 [ 59]
 [ 60]
 [ 61]
 [ 62]
 [ 63]
 [ 64]
 [ 65]
 [ 66]
 [ 67]
 [ 68]
 [ 69]
 [ 70]
 [ 71]
 [ 72]
 [ 73]
 [ 74]
 [ 75]
 [ 76]
 [ 77]
 [ 78]
 [ 79]
 [ 80]
 [ 81]
 [ 82]
 [ 83]
 [ 84]
 [ 85]
 [ 86]
 [ 87]
 [ 88]
 [ 89]
 [ 90]
 [ 91]
 [ 92]
 [ 93]
 [ 94]
 [ 95]
 [ 96]
 [ 97]
 [ 98]
 [ 99]
 [100]
 [101]
 [102]
 [103]
 [104]
 [105]
 [106]
 [107]
 [108]
 [109]
 [110]
 [111]
 [112]
 [113]
 [114]
 [115]
 [116]
 [117]
 [118]
 [119]
 [120]
 [121]
 [122]
 [123]
 [124]
 [125]
 [126]
 [127]
 [128]
 [129]
 [130]
 [131]
 [132]
 [133]
 [134]
 [135]
 [136]
 [137]
 [

In [7]:
dataset = tf.data.TFRecordDataset(record_train)


data = {
    'image/height': tf.io.FixedLenFeature([], tf.int64),
    'image/width': tf.io.FixedLenFeature([], tf.int64),
    'image/filename': tf.io.FixedLenFeature([], tf.string),
    'image/source_id': tf.io.FixedLenFeature([], tf.string),
    #'image/key/sha256':tf.io.FixedLenFeature([], tf.string),
    'image/encoded': tf.io.FixedLenFeature([], tf.string),
    #'image/encoded': tf.io.decode_png([], channels=0),
    'image/format': tf.io.FixedLenFeature([], tf.string),
    'image/object/bbox/xmin': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'image/object/bbox/xmax': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'image/object/bbox/ymin': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'image/object/bbox/ymax': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'image/object/class/text': tf.io.FixedLenSequenceFeature([], tf.string, allow_missing=True),
    'image/object/class/label': tf.io.FixedLenSequenceFeature([], tf.int64, allow_missing=True),
}

training_data = []

for element in dataset.as_numpy_iterator():
    #example = tf.train.Example.FromString(element)
    example = tf.io.parse_single_example(element, data)
    training_data = [training_data, example]
    break

example.keys()
example['image/decoded'] = tf.io.decode_image(example.get('image/encoded'))

example.keys()

dict_keys(['image/encoded', 'image/filename', 'image/format', 'image/height', 'image/object/bbox/xmax', 'image/object/bbox/xmin', 'image/object/bbox/ymax', 'image/object/bbox/ymin', 'image/object/class/label', 'image/object/class/text', 'image/source_id', 'image/width', 'image/decoded'])

In [8]:
label_map_dict = label_map_util.get_label_map_dict(class_map)

#for keys in label_map_dict.items

In [9]:
img_raw = example['image/decoded'].numpy()

#bboxes = [example['image/object/bbox/xmin'].numpy(), example['image/object/bbox/xmax'].numpy(),
#          example['image/object/bbox/ymin'].numpy(), example['image/object/bbox/ymax'].numpy()] 

example_np_xmin = example['image/object/bbox/xmin'].numpy()
example_np_xmax = example['image/object/bbox/xmax'].numpy()
example_np_ymin = example['image/object/bbox/ymin'].numpy()
example_np_ymax = example['image/object/bbox/ymax'].numpy()

example_np_bbox = np.vstack((example_np_ymin, example_np_xmin, example_np_ymax, example_np_xmax)).transpose()

example_np_labels = example['image/object/class/label'].numpy()

example_np_bbox = np.vstack((example_np_ymin, example_np_xmin, example_np_ymax, example_np_xmax)).transpose()

print(example_np_bbox)

[[0.85227275 0.14142571 0.8642045  0.14572   ]
 [0.8494318  0.16289723 0.8659091  0.16747782]
 [0.8505682  0.17921558 0.8642045  0.18436874]
 ...
 [0.54829544 0.06269682 0.6153409  0.9587747 ]
 [0.68125    0.06298311 0.74886364 0.9584884 ]
 [0.81363636 0.06241054 0.8823864  0.9582021 ]]


In [10]:
plt.figure(figsize=(10,10))
plt.imshow(img_raw_rgb)
plt.show()

NameError: name 'img_raw_rgb' is not defined

In [58]:
img_raw_rgb = np.repeat(img_raw, 3, axis=2)
img_raw_rgb.shape

(1760, 3493, 3)

In [79]:
dummy_scores = np.repeat(np.array([1.0], dtype=np.float32), example_np_labels.shape[0])

plot_detections(img_raw_rgb, example_np_bbox, example_np_labels, dummy_scores, label_map_dict)

In [73]:
example_np_labels

array([95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95,
       95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95,
       95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95,
       95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95,
       95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95,
       95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95,
       95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95,
       95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95,
       95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95,
       95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95,
       95, 95, 95, 89, 89, 89, 89, 89, 89, 89, 89, 89, 89, 89, 89, 89, 89,
       89, 89, 89, 95, 95, 95, 95,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  7,  7

In [78]:
def plot_detections(image_np,
                    boxes,
                    classes, 
                    scores, 
                    category_index, 
                    figsize=(12,16),
                    image_name=None):
    image_np_with_annotations = image_np.copy()
    viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_annotations,
        boxes,
        classes,
        scores,
        category_index,
        use_normalized_coordinates=True,
        min_score_thresh=0.8,
        max_boxes_to_draw=scores.shape[0]
    )
    if image_name:
        plt.imsave(image_name, image_np_with_annotations)
    else:
        plt.imshow(image_np_with_annotations)
        plt.show()
    
    

In [12]:
def plot_detections_from_dataset_element(element):
  dummy_scores = np.repeat(np.array([1.0], dtype=np.float32), element['image/object/class/label'].shape[0])
  label_map_dict = label_map_util.get_label_map_dict(class_map)
  image_np = element['image/decoded'].numpy().copy()
  image_np = np.repeat(image_np, 3, 2)
  boxes_np = element['image/bbox'].numpy()
  classes_np = element['image/object/class/label'].numpy()
  viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np,
                boxes=boxes_np,
                classes=classes_np,
                scores=dummy_scores,
                category_index=label_map_dict,
                use_normalized_coordinates=True,
                min_score_thresh = 0.8,
                max_boxes_to_draw=dummy_scores.shape[0]
  )
  plt.imshow(image_np)
  plt.show()



In [13]:
plot_detections_from_dataset_element(td_element)

In [24]:
dummy_scores = np.repeat(np.array([1.0], dtype=np.float32), td_element['image/object/class/label'].shape[0])
label_map_dict = label_map_util.get_label_map_dict(class_map)

td_element['image/decoded']



plot_detections(image_np=td_element['image/decoded'],
                  boxes=td_element['image/bbox'],
                  classes=td_element['image/object/class/text'],
                  scores=dummy_scores,
                  category_index=label_map_dict)

TypeError: Tensor is unhashable. Instead, use tensor.ref() as the key.